In [1]:
import pandas as pd

In [2]:
def to_list(x):
    if x == "":
        return []
    return [int(i) for i in x.split(",")]

def load_data(file):
    data = pd.read_csv(file, sep=";", names=["seen", "bought"],
                       converters={0: to_list, 1: to_list},
                       skip_blank_lines=True)
    return data

In [3]:
train = load_data("coursera_sessions_train.txt")
print train.shape
train.head()

(50000, 2)


,seen,bought
0,"[0, 1, 2, 3, 4, 5]",[]
1,"[9, 10, 11, 9, 11, 12, 9, 11]",[]
2,"[16, 17, 18, 19, 20, 21]",[]
3,"[24, 25, 26, 27, 24]",[]
4,"[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]",[]


In [4]:
test = load_data("coursera_sessions_test.txt")
print test.shape
test.head()

(50000, 2)


,seen,bought
0,"[6, 7, 8]",[]
1,"[13, 14, 15]",[]
2,"[22, 23]",[]
3,"[28, 29, 30, 31, 32, 33]",[]
4,"[40, 41]",[]


In [5]:
def calc_freq(col):
    freq = {}
    
    for (_, items) in col.iteritems():
        for item in items:
            if item not in freq:
                freq[item] = 0
            freq[item] += 1
    return freq

In [6]:
id_train_seen_freq = calc_freq(train["seen"])
id_train_bought_freq = calc_freq(train["bought"])

In [7]:
def unique(l):
    seen = set()
    seen_add = seen.add
    return [x for x in l if not (x in seen or seen_add(x))]

def intersect(a, b):
    return list(set(a) & set(b))

In [8]:
def avg_precision(k, data, freqs):
    precision = 0
    total = 0
    
    for row in data.iterrows():
        row = row[1]
        if len(row['bought']) == 0:
            continue
        r = sorted(unique(row['seen']), key = lambda x: freqs.get(x, 0), reverse=True)[0: min(k, len(row['seen']))]
        precision += float(len(intersect(r, row['bought']))) / k
        total += 1
    return precision / total

def avg_recall(k, data, freqs):
    recall = 0
    total = 0

    for row in data.iterrows():
        row = row[1]
        if len(row['bought']) == 0:
            continue
        r = sorted(unique(row['seen']), key = lambda x: freqs.get(x, 0), reverse=True)[0: min(k, len(row['seen']))]
        recall += float(len(intersect(r, row['bought']))) / len(row['bought'])
        total += 1
    return recall / total

In [9]:
def save_answer(f, r1, p1, r5, p5):
    with open(f, "w") as f:
        f.write("%.2f %.2f %.2f %.2f" % (r1, p1, r5, p5))

In [10]:
save_answer("answer1.txt",
            avg_recall(1, train, id_train_seen_freq),
            avg_precision(1, train, id_train_seen_freq),
            avg_recall(5, train, id_train_seen_freq),
            avg_precision(5, train, id_train_seen_freq))

In [11]:
save_answer("answer2.txt",
            avg_recall(1, test, id_train_seen_freq),
            avg_precision(1, test, id_train_seen_freq),
            avg_recall(5, test, id_train_seen_freq),
            avg_precision(5, test, id_train_seen_freq))

In [12]:
save_answer("answer3.txt",
            avg_recall(1, train, id_train_bought_freq),
            avg_precision(1, train, id_train_bought_freq),
            avg_recall(5, train, id_train_bought_freq),
            avg_precision(5, train, id_train_bought_freq))

In [13]:
save_answer("answer4.txt",
            avg_recall(1, test, id_train_bought_freq),
            avg_precision(1, test, id_train_bought_freq),
            avg_recall(5, test, id_train_bought_freq),
            avg_precision(5, test, id_train_bought_freq))